In [ ]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

### Request to access the webpage and creating the soup (contains the html of entire page)

In [ ]:
url='https://www.imdb.com/chart/top?ref_=nv_mv_250'
html=urlopen(url)
soup=BeautifulSoup(html,'lxml')
titles=soup.findAll('td',{'class':'titleColumn'})

### Creating file to store data

In [ ]:
filename1='IMDB1.csv'
f=open(filename1,'w')
head='Movie_name,Release_Year,link,Rating,Number_of_User,RunTime,Censor_Board_rating,Release_Date,Genre,Summary,Director,Writers,Star,Keywords,Budget,Opening_weekend,Gross_Col,Worldwide_Gross,Production\n'
f.write(head)

In [ ]:
titles=soup.findAll('td',{'class':'titleColumn'})
for title in range(len(titles)):
    link = 'https://www.imdb.com'+titles[title].a['href']
    
    l=urlopen(link)
    soup1=BeautifulSoup(l,'lxml')
    
    ny=soup1.h1.text.split(sep='\xa0')
    Movie_name = ny[0]   #Movie Name
    Release_Year = ny[1].split('(')[1].split(')')[0] #Year
    
    rating=soup1.findAll('div',{'class':'ratingValue'})
    r=rating[0].strong['title'].split()
    Rating=r[0]  #rating
    Number_of_User=r[3].replace(',','') #user_rating
    
    cb1=soup1.findAll('div',{'class':'subtext'})
    RunTime=(cb1[0].time.text.strip())#runttime
    Censor_Board_rating=(cb1[0].text.strip().split('|')[0].strip()) #Censor Board Rating
    
    cb=soup1.findAll('div',{'class':'title_wrapper'})
    g=cb[0].div.findAll('a')

    genre=''
    year=''
    for i in range(len(g)):
        if '\n' not in g[i].text:
            genre+=g[i].text
            if i!=len(g)-2:
                genre+='|'
        else:
            year=g[i].text
    Release_Date=(year.replace('\n','')) #Released date
    Genre=(genre) # Genre
    
    s=soup1.findAll('div',{'class':'summary_text'})
    Summary=s[0].text.strip() #Summary
    
    d=soup1.findAll('div',{'class':'credit_summary_item'})
    di=d[0].text.split('\n')
    Director=di[2] #Director
    
    w=d[1].findAll('a')
    Writer=''
    for i in range(len(w)):
        if '1' in w[i].text:
            break
        else:
            Writer+=w[i].text
            if i!=len(w)-2:
                Writer+='|'
    Writers=Writer #Writers
    
    
    s=d[2].findAll('a')
    Stars=''
    for i in range(len(s)):
        if 'See' in s[i].text:
            break
        else:
            Stars+=s[i].text
            if i!=len(s)-2:
                Stars+='|'
    Star=Stars #Stars
    
    
    key=soup1.findAll('div',{'class':'see-more inline canwrap'})
    ke=key[0].findAll('span',class_='itemprop')
    keyword=''
    for i in range(len(ke)):
        keyword+=ke[i].text
        if i!=len(ke)-1:
            keyword+='|'

    Keywords=keyword #keywords
    
    de=soup1.findAll('div',{'class':'article','id':'titleDetails'})
    det=de[0].findAll('div',{'class':'txt-block'})
    detail=''
    var=['Budget','Gross USA','Opening Weekend USA','Cumulative Worldwide Gross']
    for i in range(len(det)):
        for j in range(len(var)): 
            deta=det[i].text.strip().split(':')
            if deta[0] in var[j]:
                detail+=var[j]+':'+ deta[1].strip() + '|'  
    dl=detail.split('|')
    ld=len(detail.split('|'))
    for i in range(ld):
        try:
            if 'Budget' in dl[i]:
                Budget=(dl[i].split()[0].split(':')[1].replace(',',''))
            elif 'Gross USA' in dl[i]:
                Gross_Col=(dl[i].split()[1].split(':')[1].replace(',',''))
            elif 'Opening Weekend USA' in dl[i]:
                Opening_weekend=(dl[i].split()[2].split(':')[1].replace(',',''))
            else:
                Worldwide_Gross=(dl[i].split(':')[1].replace(',',''))
        except: IndexError 
    for i in range(len(det)):
        deta=det[i].text.strip().split(':')
        if 'Production' in deta[0]:
            Production=(deta[1].split('\n')[1].replace(',',' &'))
    

    
    f.write(Movie_name.replace(',',';') +','+ Release_Year +','+ link +','+ Rating +','+ Number_of_User+ ','+RunTime+','+Censor_Board_rating+','+Release_Date+','+Genre+','+Summary.replace(',',':')+','+Director+','+Writers+','+Star+','+Keywords+','+Budget+','+Opening_weekend+','+Gross_Col+','+Worldwide_Gross+','+Production+'\n')

In [ ]:
f.close()